In [114]:
from sklearn import svm
import numpy as np
import os
import numpy
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
def SVM(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)

# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
#     print(collections.Counter(ArousalFile))
#     print(collections.Counter(ValenceFile))
#     print(np.mean(ValenceFile))
#     print(np.mean(ArousalFile))
    
# SVM Classifier Valence
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
#     print(cm)
#     print("Accuracy score of Valence ")
#     print(accuracy_score(y_test, y_predict)*100)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    	
# SVM Classifier Arousal
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    cm = confusion_matrix(z_test, z_predict)
#     print(z_test)
#     print(z_predict)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100]

    
# print("$$$$$$$Arousal$$$$$$$")

def SVMCombined(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100]



def SVMHS(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)):
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
        
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100]

def Run100(UserName,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    for i in range (100):
        svmpca=SVM(UserName,Window,StartTime,EndTime)
        arousal+=[svmpca[0]]
        valence+=[svmpca[1]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence)];
    

    
def RunCombined100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    for i in range (100):
        svm=SVMCombined(UserName,Window,StartTime,EndTime)
        Combined+=svm
    #print("Combined "+str(np.mean(Combined)))
    print(collections.Counter(Combined))
    return np.mean(Combined)
    
def RunHS100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    for i in range (100):
        svm=SVMHS(UserName,Window,StartTime,EndTime)
        Combined+=svm
    #print("Happy and Sad "+str(np.mean(Combined)))
    print(collections.Counter(Combined))
    return np.mean(Combined);

    
if __name__ == "__main__":
    HS=0;
    Combined=0;
    Valence=0
    Arousal=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HS+=var
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        Combined+=var1
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var1)
        Valence+=var2[0]
        Arousal+=var2[1]
    for i in range(10,31):
        print("s"+str(i))
        #Happy and Sad
        var =RunHS100("s"+str(i))
        print("Happy Sad")
        print(var)
        HS+=var
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        Combined+=var1
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var1)
        Valence+=var2[0]
        Arousal+=var2[1]
    print("Average happy sad for first 30 "+str(HS/30))
    print("Average Combined sad for first 30 "+str(Combined/30))
    print("Average Valence for first 30 "+str(Valence/30))
    print("Average Arousal for first 30 "+str(Arousal/30))

s01
Counter({40.0: 42, 60.0: 34, 80.0: 13, 20.0: 10, 0.0: 1})
Happy Sad
49.6
Counter({25.0: 30, 37.5: 27, 12.5: 22, 50.0: 12, 62.5: 4, 0.0: 3, 75.0: 2})
Combined Emotions
30.375
Arousal 61.0
Counter({75.0: 27, 62.5: 25, 50.0: 21, 37.5: 13, 87.5: 10, 25.0: 4})
Valence 36.875
Counter({37.5: 36, 50.0: 26, 25.0: 22, 12.5: 9, 62.5: 6, 0.0: 1})
Arousal Valence
30.375
s02
Counter({80.0: 46, 100.0: 45, 60.0: 9})
Happy Sad
87.2
Counter({50.0: 29, 37.5: 24, 62.5: 23, 25.0: 12, 75.0: 9, 12.5: 2, 0.0: 1})
Combined Emotions
47.875
Arousal 65.5
Counter({62.5: 34, 75.0: 26, 50.0: 17, 87.5: 12, 37.5: 6, 100.0: 3, 25.0: 2})
Valence 68.5
Counter({75.0: 36, 62.5: 30, 87.5: 16, 50.0: 13, 37.5: 3, 100.0: 1, 12.5: 1})
Arousal Valence
47.875
s03
Counter({33.33333333333333: 55, 0.0: 24, 66.66666666666666: 21})
Happy Sad
32.33333333333333
Counter({62.5: 36, 50.0: 26, 75.0: 17, 37.5: 13, 87.5: 4, 25.0: 3, 100.0: 1})
Combined Emotions
58.375
Arousal 75.0
Counter({75.0: 45, 87.5: 25, 62.5: 19, 50.0: 7, 100.0: 4})

Arousal 74.0
Counter({87.5: 32, 75.0: 31, 62.5: 24, 50.0: 8, 100.0: 3, 37.5: 2})
Valence 47.125
Counter({50.0: 36, 37.5: 27, 62.5: 18, 25.0: 8, 75.0: 7, 12.5: 4})
Arousal Valence
49.25
s23
Counter({60.0: 45, 80.0: 30, 40.0: 16, 20.0: 6, 100.0: 3})
Happy Sad
61.6
Counter({37.5: 29, 25.0: 26, 12.5: 19, 50.0: 16, 62.5: 7, 0.0: 3})
Combined Emotions
32.125
Arousal 56.125
Counter({50.0: 33, 62.5: 27, 75.0: 18, 37.5: 15, 25.0: 4, 87.5: 3})
Valence 78.5
Counter({87.5: 34, 75.0: 31, 62.5: 20, 100.0: 11, 50.0: 4})
Arousal Valence
32.125
s24
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
Counter({100.0: 100})
Happy Sad
100.0
Counter({50.0: 36, 37.5: 34, 62.5: 14, 25.0: 9, 12.5: 3, 75.0: 3, 0.0: 1})
Combined Emotions
44.375
Arousal 87.125
Counter({87.5: 44, 100.0: 28, 75.0: 25, 62.5: 3})
Valence 40.125
Counter({37.5: 43, 50.0: 32, 25.0: 15, 62.5: 6, 12.5: 